# DAY 1 - NOTEBOOK 1: SPARK ARCHITECTURE & DATAFRAME BASICS

---

## 🎯 Learning Objectives:
1. Understand Spark Architecture
2. Create DataFrames (3 methods)
3. Basic operations (select, filter, show)
4. Schema operations
5. Column operations

---

## PART 1: UNDERSTANDING SPARK ARCHITECTURE

```
┌─────────────────────────────────────────────────────────────────┐
│                    SPARK CLUSTER ARCHITECTURE                   │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│                    ┌──────────────┐                             │
│                    │   DRIVER     │  ← Your Jupyter Notebook    │
│                    │  (jupyter)   │                             │
│                    └──────┬───────┘                             │
│                           │                                     │
│                           │ Submit Job                          │
│                           ▼                                     │
│                    ┌──────────────┐                             │
│                    │CLUSTER MANAGER│ ← Spark Master             │
│                    │(spark-master) │                            │
│                    └──────┬────────┘                            │
│                           │                                     │
│                           │ Allocate Resources                  │
│                           ▼                                     │
│         ┌─────────────────────────────────────┐                 │
│         │          EXECUTORS                  │                 │
│         │  ┌──────────┐    ┌──────────┐      │                 │
│         │  │ Worker 1 │    │ Worker 2 │      │                 │
│         │  │ ┌──────┐ │    │ ┌──────┐ │      │                 │
│         │  │ │Task 1│ │    │ │Task 3│ │      │                 │
│         │  │ │Task 2│ │    │ │Task 4│ │      │                 │
│         │  │ └──────┘ │    │ └──────┘ │      │                 │
│         │  └──────────┘    └──────────┘      │                 │
│         └─────────────────────────────────────┘                 │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Key Components:

| Component | Role | Example |
|-----------|------|----------|
| **Driver** | Orchestrates job | Jupyter Notebook |
| **Cluster Manager** | Manages resources | Spark Master |
| **Executors** | Execute tasks | Workers (2 workers) |
| **Tasks** | Smallest work unit | Process 1 partition |

### Data Flow:
1. Driver creates execution plan
2. Cluster Manager allocates resources
3. Executors receive tasks
4. Tasks process data in parallel
5. Results return to Driver

---

## PART 2: CREATE SPARK SESSION

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Create session with optimized config
spark = SparkSession.builder \
    .appName("Day1-DataFrameBasics") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "512m") \
    .config("spark.executor.cores", "1") \
    .config("spark.cores.max", "2") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.shuffle.service.enabled", "false") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

print("✅ Spark Session Created!")
print(f"   Version: {spark.version}")
print(f"   Master: {spark.sparkContext.master}")
print(f"   App ID: {spark.sparkContext.applicationId}")
print(f"   Default Parallelism: {spark.sparkContext.defaultParallelism}")

✅ Spark Session Created!
   Version: 3.5.1
   Master: spark://spark-master:7077
   App ID: app-20251222152137-0000
   Default Parallelism: 2


---

## PART 3: CREATING DATAFRAMES (3 METHODS)

### Method 1: From Python List of Tuples

In [3]:
data = [
    (1, "Alice", 25, "Engineering", 75000),
    (2, "Bob", 30, "Sales", 65000),
    (3, "Charlie", 35, "Engineering", 85000),
    (4, "David", 28, "HR", 55000),
    (5, "Eve", 32, "Sales", 70000)
]

df = spark.createDataFrame(data, ["id", "name", "age", "department", "salary"])

print("✅ Created DataFrame from list of tuples")
print("\n📊 Display data:")
df.show()

print("📋 Schema:")
df.printSchema()

✅ Created DataFrame from list of tuples

📊 Display data:
+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  1|  Alice| 25|Engineering| 75000|
|  2|    Bob| 30|      Sales| 65000|
|  3|Charlie| 35|Engineering| 85000|
|  4|  David| 28|         HR| 55000|
|  5|    Eve| 32|      Sales| 70000|
+---+-------+---+-----------+------+

📋 Schema:
root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)



### Method 2: From List of Dictionaries

In [4]:
data_dict = [
    {"id": 1, "name": "Alice", "age": 25, "city": "New York"},
    {"id": 2, "name": "Bob", "age": 30, "city": "London"},
    {"id": 3, "name": "Charlie", "age": 35, "city": "Paris"}
]

df_dict = spark.createDataFrame(data_dict)

print("✅ Created DataFrame from list of dictionaries")
print("\n📊 Display data:")
df_dict.show()

✅ Created DataFrame from list of dictionaries

📊 Display data:
+---+--------+---+-------+
|age|    city| id|   name|
+---+--------+---+-------+
| 25|New York|  1|  Alice|
| 30|  London|  2|    Bob|
| 35|   Paris|  3|Charlie|
+---+--------+---+-------+



### Method 3: With Explicit Schema (RECOMMENDED for Production)

In [7]:
# Define schema
schema = StructType([
    StructField("id", IntegerType(), nullable=False),
    StructField("name", StringType(), nullable=False),
    StructField("age", IntegerType(), nullable=True),
    StructField("department", StringType(), nullable=True),
    StructField("salary", IntegerType(), nullable=True)
])

df_schema = spark.createDataFrame(data, schema)

print("✅ Created DataFrame with explicit schema")
print("\n📋 Schema with constraints:")
df_schema.printSchema()

print("""
💡 Why use explicit schema?
   ✅ Type safety (prevent errors)
   ✅ Better performance (no inference)
   ✅ Nullable constraints
   ✅ Documentation
""")

✅ Created DataFrame with explicit schema

📋 Schema with constraints:
root
 |-- id: integer (nullable = false)
 |-- name: string (nullable = false)
 |-- age: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: integer (nullable = true)


💡 Why use explicit schema?
   ✅ Type safety (prevent errors)
   ✅ Better performance (no inference)
   ✅ Nullable constraints
   ✅ Documentation



---

## PART 4: BASIC DATAFRAME OPERATIONS

### 4.1: Inspecting Data

In [8]:
print("📌 show() - Display data")
df.show()

📌 show() - Display data
+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  1|  Alice| 25|Engineering| 75000|
|  2|    Bob| 30|      Sales| 65000|
|  3|Charlie| 35|Engineering| 85000|
|  4|  David| 28|         HR| 55000|
|  5|    Eve| 32|      Sales| 70000|
+---+-------+---+-----------+------+



In [9]:
print("📌 show(n) - Display first n rows")
df.show(3)

📌 show(n) - Display first n rows


+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  1|  Alice| 25|Engineering| 75000|
|  2|    Bob| 30|      Sales| 65000|
|  3|Charlie| 35|Engineering| 85000|
+---+-------+---+-----------+------+
only showing top 3 rows



In [10]:
print("📌 show(truncate=False) - Don't truncate long strings")
df.show(truncate=False)

📌 show(truncate=False) - Don't truncate long strings
+---+-------+---+-----------+------+
|id |name   |age|department |salary|
+---+-------+---+-----------+------+
|1  |Alice  |25 |Engineering|75000 |
|2  |Bob    |30 |Sales      |65000 |
|3  |Charlie|35 |Engineering|85000 |
|4  |David  |28 |HR         |55000 |
|5  |Eve    |32 |Sales      |70000 |
+---+-------+---+-----------+------+



In [11]:
print("📌 printSchema() - Display schema")
df.printSchema()

📌 printSchema() - Display schema
root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)



In [12]:
print("📌 columns - Get column names")
print(f"Columns: {df.columns}")

📌 columns - Get column names
Columns: ['id', 'name', 'age', 'department', 'salary']


In [13]:
print("📌 dtypes - Get data types")
print(f"Data types: {df.dtypes}")

📌 dtypes - Get data types
Data types: [('id', 'bigint'), ('name', 'string'), ('age', 'bigint'), ('department', 'string'), ('salary', 'bigint')]


In [14]:
print("📌 count() - Count rows")
print(f"Total rows: {df.count()}")

📌 count() - Count rows
Total rows: 5


In [15]:
print("📌 describe() - Summary statistics")
df.describe().show()

📌 describe() - Summary statistics


25/12/22 15:23:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-----+------------------+-----------+------------------+
|summary|                id| name|               age| department|            salary|
+-------+------------------+-----+------------------+-----------+------------------+
|  count|                 5|    5|                 5|          5|                 5|
|   mean|               3.0| NULL|              30.0|       NULL|           70000.0|
| stddev|1.5811388300841898| NULL|3.8078865529319548|       NULL|11180.339887498949|
|    min|                 1|Alice|                25|Engineering|             55000|
|    max|                 5|  Eve|                35|      Sales|             85000|
+-------+------------------+-----+------------------+-----------+------------------+



In [16]:
print("📌 first() - Get first row")
print(f"First row: {df.first()}")

📌 first() - Get first row
First row: Row(id=1, name='Alice', age=25, department='Engineering', salary=75000)


In [17]:
print("📌 take(n) - Get first n rows as list")
print(f"First 2 rows: {df.take(2)}")

📌 take(n) - Get first n rows as list
First 2 rows: [Row(id=1, name='Alice', age=25, department='Engineering', salary=75000), Row(id=2, name='Bob', age=30, department='Sales', salary=65000)]


In [18]:
print("📌 head(n) - Same as take(n)")
print(f"First 2 rows: {df.head(2)}")

📌 head(n) - Same as take(n)
First 2 rows: [Row(id=1, name='Alice', age=25, department='Engineering', salary=75000), Row(id=2, name='Bob', age=30, department='Sales', salary=65000)]


In [19]:
print("⚠️  collect() - Get ALL rows (USE WITH CAUTION!)")
print("    Only use on small datasets!")
rows = df.collect()
print(f"    Collected {len(rows)} rows")
print(f"    First row: {rows[0]}")

⚠️  collect() - Get ALL rows (USE WITH CAUTION!)
    Only use on small datasets!
    Collected 5 rows
    First row: Row(id=1, name='Alice', age=25, department='Engineering', salary=75000)


### 4.2: Selecting Columns

In [20]:
print("📌 Select single column")
df.select("name").show()

📌 Select single column
+-------+
|   name|
+-------+
|  Alice|
|    Bob|
|Charlie|
|  David|
|    Eve|
+-------+



In [21]:
print("📌 Select multiple columns")
df.select("name", "age", "salary").show()

📌 Select multiple columns
+-------+---+------+
|   name|age|salary|
+-------+---+------+
|  Alice| 25| 75000|
|    Bob| 30| 65000|
|Charlie| 35| 85000|
|  David| 28| 55000|
|    Eve| 32| 70000|
+-------+---+------+



In [22]:
print("📌 Select with col() function")
df.select(col("name"), col("age")).show()

📌 Select with col() function
+-------+---+
|   name|age|
+-------+---+
|  Alice| 25|
|    Bob| 30|
|Charlie| 35|
|  David| 28|
|    Eve| 32|
+-------+---+



In [23]:
print("📌 Select all columns")
df.select("*").show()

📌 Select all columns
+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  1|  Alice| 25|Engineering| 75000|
|  2|    Bob| 30|      Sales| 65000|
|  3|Charlie| 35|Engineering| 85000|
|  4|  David| 28|         HR| 55000|
|  5|    Eve| 32|      Sales| 70000|
+---+-------+---+-----------+------+



In [24]:
print("📌 Select with expressions")
df.select(
    col("name"),
    col("salary"),
    (col("salary") * 1.1).alias("salary_with_10pct_raise")
).show()

📌 Select with expressions
+-------+------+-----------------------+
|   name|salary|salary_with_10pct_raise|
+-------+------+-----------------------+
|  Alice| 75000|                82500.0|
|    Bob| 65000|                71500.0|
|Charlie| 85000|      93500.00000000001|
|  David| 55000|      60500.00000000001|
|    Eve| 70000|                77000.0|
+-------+------+-----------------------+



In [25]:
print("📌 Select with column renaming")
df.select(
    col("name").alias("employee_name"),
    col("department").alias("dept")
).show()

📌 Select with column renaming
+-------------+-----------+
|employee_name|       dept|
+-------------+-----------+
|        Alice|Engineering|
|          Bob|      Sales|
|      Charlie|Engineering|
|        David|         HR|
|          Eve|      Sales|
+-------------+-----------+



### 4.3: Filtering Rows

In [26]:
print("📌 Filter: age > 28")
df.filter(col("age") > 28).show()

📌 Filter: age > 28
+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  2|    Bob| 30|      Sales| 65000|
|  3|Charlie| 35|Engineering| 85000|
|  5|    Eve| 32|      Sales| 70000|
+---+-------+---+-----------+------+



In [27]:
print("📌 Filter: Multiple conditions (AND)")
df.filter((col("age") > 28) & (col("salary") > 60000)).show()

📌 Filter: Multiple conditions (AND)
+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  2|    Bob| 30|      Sales| 65000|
|  3|Charlie| 35|Engineering| 85000|
|  5|    Eve| 32|      Sales| 70000|
+---+-------+---+-----------+------+



In [28]:
print("📌 Filter: Multiple conditions (OR)")
df.filter((col("department") == "Sales") | (col("department") == "HR")).show()

📌 Filter: Multiple conditions (OR)
+---+-----+---+----------+------+
| id| name|age|department|salary|
+---+-----+---+----------+------+
|  2|  Bob| 30|     Sales| 65000|
|  4|David| 28|        HR| 55000|
|  5|  Eve| 32|     Sales| 70000|
+---+-----+---+----------+------+



In [29]:
print("📌 Filter: SQL-like syntax")
df.filter("age > 28 AND salary > 60000").show()

📌 Filter: SQL-like syntax
+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  2|    Bob| 30|      Sales| 65000|
|  3|Charlie| 35|Engineering| 85000|
|  5|    Eve| 32|      Sales| 70000|
+---+-------+---+-----------+------+



In [30]:
print("📌 Filter: Using isin()")
df.filter(col("department").isin("Sales", "Engineering")).show()

📌 Filter: Using isin()
+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  1|  Alice| 25|Engineering| 75000|
|  2|    Bob| 30|      Sales| 65000|
|  3|Charlie| 35|Engineering| 85000|
|  5|    Eve| 32|      Sales| 70000|
+---+-------+---+-----------+------+



In [31]:
print("📌 Filter: Pattern matching with like()")
df.filter(col("name").like("A%")).show()  # Names starting with 'A'

📌 Filter: Pattern matching with like()
+---+-----+---+-----------+------+
| id| name|age| department|salary|
+---+-----+---+-----------+------+
|  1|Alice| 25|Engineering| 75000|
+---+-----+---+-----------+------+



In [32]:
print("📌 Filter: NOT NULL")
df.filter(col("age").isNotNull()).show()

📌 Filter: NOT NULL
+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  1|  Alice| 25|Engineering| 75000|
|  2|    Bob| 30|      Sales| 65000|
|  3|Charlie| 35|Engineering| 85000|
|  4|  David| 28|         HR| 55000|
|  5|    Eve| 32|      Sales| 70000|
+---+-------+---+-----------+------+



In [33]:
print("📌 Filter: IS NULL")
# Add a row with NULL
df_with_null = df.union(spark.createDataFrame([(6, "Frank", None, "IT", 80000)], df.columns))
df_with_null.filter(col("age").isNull()).show()

📌 Filter: IS NULL


PySparkValueError: [CANNOT_DETERMINE_TYPE] Some of types cannot be determined after inferring.

### 4.4: Adding & Modifying Columns

In [34]:
print("📌 Add new column: annual_salary")
df_with_annual = df.withColumn("annual_salary", col("salary") * 12)
df_with_annual.show()

📌 Add new column: annual_salary
+---+-------+---+-----------+------+-------------+
| id|   name|age| department|salary|annual_salary|
+---+-------+---+-----------+------+-------------+
|  1|  Alice| 25|Engineering| 75000|       900000|
|  2|    Bob| 30|      Sales| 65000|       780000|
|  3|Charlie| 35|Engineering| 85000|      1020000|
|  4|  David| 28|         HR| 55000|       660000|
|  5|    Eve| 32|      Sales| 70000|       840000|
+---+-------+---+-----------+------+-------------+



In [35]:
print("📌 Add multiple columns (chaining)")
df_enhanced = df \
    .withColumn("annual_salary", col("salary") * 12) \
    .withColumn("age_in_5_years", col("age") + 5) \
    .withColumn("is_senior", col("age") > 30)
df_enhanced.show()

📌 Add multiple columns (chaining)
+---+-------+---+-----------+------+-------------+--------------+---------+
| id|   name|age| department|salary|annual_salary|age_in_5_years|is_senior|
+---+-------+---+-----------+------+-------------+--------------+---------+
|  1|  Alice| 25|Engineering| 75000|       900000|            30|    false|
|  2|    Bob| 30|      Sales| 65000|       780000|            35|    false|
|  3|Charlie| 35|Engineering| 85000|      1020000|            40|     true|
|  4|  David| 28|         HR| 55000|       660000|            33|    false|
|  5|    Eve| 32|      Sales| 70000|       840000|            37|     true|
+---+-------+---+-----------+------+-------------+--------------+---------+



In [36]:
print("📌 Modify existing column")
df_modified = df.withColumn("salary", col("salary") / 1000)
print("Salary in thousands:")
df_modified.show()

📌 Modify existing column
Salary in thousands:
+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  1|  Alice| 25|Engineering|  75.0|
|  2|    Bob| 30|      Sales|  65.0|
|  3|Charlie| 35|Engineering|  85.0|
|  4|  David| 28|         HR|  55.0|
|  5|    Eve| 32|      Sales|  70.0|
+---+-------+---+-----------+------+



In [37]:
print("📌 Rename column")
df_renamed = df.withColumnRenamed("salary", "monthly_salary")
df_renamed.show()

📌 Rename column
+---+-------+---+-----------+--------------+
| id|   name|age| department|monthly_salary|
+---+-------+---+-----------+--------------+
|  1|  Alice| 25|Engineering|         75000|
|  2|    Bob| 30|      Sales|         65000|
|  3|Charlie| 35|Engineering|         85000|
|  4|  David| 28|         HR|         55000|
|  5|    Eve| 32|      Sales|         70000|
+---+-------+---+-----------+--------------+



In [38]:
print("📌 Drop column")
df_dropped = df.drop("department")
df_dropped.show()

📌 Drop column
+---+-------+---+------+
| id|   name|age|salary|
+---+-------+---+------+
|  1|  Alice| 25| 75000|
|  2|    Bob| 30| 65000|
|  3|Charlie| 35| 85000|
|  4|  David| 28| 55000|
|  5|    Eve| 32| 70000|
+---+-------+---+------+



In [39]:
print("📌 Drop multiple columns")
df_dropped_multi = df.drop("department", "salary")
df_dropped_multi.show()

📌 Drop multiple columns
+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 25|
|  2|    Bob| 30|
|  3|Charlie| 35|
|  4|  David| 28|
|  5|    Eve| 32|
+---+-------+---+



### 4.5: Sorting

In [40]:
print("📌 Sort by age (ascending)")
df.orderBy("age").show()

📌 Sort by age (ascending)
+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  1|  Alice| 25|Engineering| 75000|
|  4|  David| 28|         HR| 55000|
|  2|    Bob| 30|      Sales| 65000|
|  5|    Eve| 32|      Sales| 70000|
|  3|Charlie| 35|Engineering| 85000|
+---+-------+---+-----------+------+



In [41]:
print("📌 Sort by salary (descending)")
df.orderBy(col("salary").desc()).show()

📌 Sort by salary (descending)
+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  3|Charlie| 35|Engineering| 85000|
|  1|  Alice| 25|Engineering| 75000|
|  5|    Eve| 32|      Sales| 70000|
|  2|    Bob| 30|      Sales| 65000|
|  4|  David| 28|         HR| 55000|
+---+-------+---+-----------+------+



In [42]:
print("📌 Sort by multiple columns")
df.orderBy("department", col("salary").desc()).show()

📌 Sort by multiple columns
+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  3|Charlie| 35|Engineering| 85000|
|  1|  Alice| 25|Engineering| 75000|
|  4|  David| 28|         HR| 55000|
|  5|    Eve| 32|      Sales| 70000|
|  2|    Bob| 30|      Sales| 65000|
+---+-------+---+-----------+------+



In [43]:
print("📌 Sort with asc() and desc() explicitly")
df.orderBy(col("age").asc(), col("salary").desc()).show()

📌 Sort with asc() and desc() explicitly
+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  1|  Alice| 25|Engineering| 75000|
|  4|  David| 28|         HR| 55000|
|  2|    Bob| 30|      Sales| 65000|
|  5|    Eve| 32|      Sales| 70000|
|  3|Charlie| 35|Engineering| 85000|
+---+-------+---+-----------+------+



### 4.6: Distinct & Drop Duplicates

In [44]:
# Create data with duplicates
data_with_dups = data + [
    (6, "Alice", 25, "Engineering", 75000),  # Duplicate
    (7, "Bob", 30, "Sales", 65000)  # Duplicate
]
df_dups = spark.createDataFrame(data_with_dups, ["id", "name", "age", "department", "salary"])

print("📌 Original data with duplicates")
df_dups.show()

📌 Original data with duplicates
+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  1|  Alice| 25|Engineering| 75000|
|  2|    Bob| 30|      Sales| 65000|
|  3|Charlie| 35|Engineering| 85000|
|  4|  David| 28|         HR| 55000|
|  5|    Eve| 32|      Sales| 70000|
|  6|  Alice| 25|Engineering| 75000|
|  7|    Bob| 30|      Sales| 65000|
+---+-------+---+-----------+------+



In [45]:
print("📌 Get distinct rows")
df_dups.distinct().show()

📌 Get distinct rows
+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  4|  David| 28|         HR| 55000|
|  5|    Eve| 32|      Sales| 70000|
|  7|    Bob| 30|      Sales| 65000|
|  6|  Alice| 25|Engineering| 75000|
|  3|Charlie| 35|Engineering| 85000|
|  1|  Alice| 25|Engineering| 75000|
|  2|    Bob| 30|      Sales| 65000|
+---+-------+---+-----------+------+



In [46]:
print("📌 Drop duplicates based on specific columns")
df_dups.dropDuplicates(["name", "age"]).show()

📌 Drop duplicates based on specific columns
+---+-------+---+-----------+------+
| id|   name|age| department|salary|
+---+-------+---+-----------+------+
|  1|  Alice| 25|Engineering| 75000|
|  2|    Bob| 30|      Sales| 65000|
|  3|Charlie| 35|Engineering| 85000|
|  4|  David| 28|         HR| 55000|
|  5|    Eve| 32|      Sales| 70000|
+---+-------+---+-----------+------+



In [47]:
print("📌 Get distinct values of a column")
df.select("department").distinct().show()

📌 Get distinct values of a column
+-----------+
| department|
+-----------+
|Engineering|
|      Sales|
|         HR|
+-----------+



---

## PART 5: WORKING WITH LARGER DATASET

In [48]:
# Generate larger dataset using Faker
from faker import Faker
import random

fake = Faker()
Faker.seed(42)
random.seed(42)

print("📌 Generating 1000 rows of employee data...")

large_data = []
departments = ["Engineering", "Sales", "HR", "Marketing", "Finance", "IT", "Operations"]

for i in range(1, 1001):
    large_data.append({
        "id": i,
        "name": fake.name(),
        "age": random.randint(22, 65),
        "department": random.choice(departments),
        "salary": random.randint(40000, 150000),
        "city": fake.city(),
        "country": fake.country(),
        "email": fake.email(),
        "join_date": fake.date_between(start_date="-10y", end_date="today").isoformat()
    })

df_large = spark.createDataFrame(large_data)

print(f"✅ Created dataset with {df_large.count()} rows")
print(f"   Partitions: {df_large.rdd.getNumPartitions()}")

📌 Generating 1000 rows of employee data...
✅ Created dataset with 1000 rows
   Partitions: 2


In [49]:
print("📊 Sample data:")
df_large.show(10)

📊 Sample data:
+---+---------------+----------------+-----------+--------------------+---+----------+-----------------+------+
|age|           city|         country| department|               email| id| join_date|             name|salary|
+---+---------------+----------------+-----------+--------------------+---+----------+-----------------+------+
| 62|      East Jill|     Philippines|Engineering|garzaanthony@exam...|  1|2018-02-27|     Allison Hill| 43278|
| 39|  Robinsonshire|            Fiji|      Sales|blakeerik@example...|  2|2019-05-17|  Alyssa Gonzalez| 69256|
| 30|South Colinstad|     Netherlands|         IT|lindsay78@example...|  3|2025-09-14|     Tyler Rogers| 53434|
| 65|    Barbaraland|           Aruba|         IT|kendragalloway@ex...|  4|2024-08-21|    Juan Calderon|111482|
| 27|   West Michael|        Paraguay|    Finance|jacqueline19@exam...|  5|2017-09-07|    Jesse Flowers| 95302|
| 24|      Lake Mark|      Seychelles|Engineering|perezantonio@exam...|  6|2019-12-26|   

In [50]:
print("📊 Schema:")
df_large.printSchema()

📊 Schema:
root
 |-- age: long (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- department: string (nullable = true)
 |-- email: string (nullable = true)
 |-- id: long (nullable = true)
 |-- join_date: string (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: long (nullable = true)



In [51]:
print("📊 Summary statistics:")
df_large.describe().show()

📊 Summary statistics:
+-------+------------------+-----------+-----------+-----------+--------------------+-----------------+----------+----------------+------------------+
|summary|               age|       city|    country| department|               email|               id| join_date|            name|            salary|
+-------+------------------+-----------+-----------+-----------+--------------------+-----------------+----------+----------------+------------------+
|  count|              1000|       1000|       1000|       1000|                1000|             1000|      1000|            1000|              1000|
|   mean|            43.856|       NULL|       NULL|       NULL|                NULL|            500.5|      NULL|            NULL|         95900.788|
| stddev|12.680222870756339|       NULL|       NULL|       NULL|                NULL|288.8194360957494|      NULL|            NULL|30966.077697393597|
|    min|                22|  Adamburgh|Afghanistan|Engineering|  abrown

### 5.1: Sampling

In [52]:
print("📌 Random sample: 10% (without replacement)")
df_sample = df_large.sample(fraction=0.1, seed=42)
print(f"Sample size: {df_sample.count()} rows")
df_sample.show(5)

📌 Random sample: 10% (without replacement)
Sample size: 95 rows
+---+--------------+---------------+-----------+--------------------+---+----------+-----------------+------+
|age|          city|        country| department|               email| id| join_date|             name|salary|
+---+--------------+---------------+-----------+--------------------+---+----------+-----------------+------+
| 60|       Coxberg|         Malawi|Engineering|  ddavis@example.org|  8|2017-07-02| Nicole Patterson|113563|
| 28|East Jamesside|       Kiribati|         HR|robertramirez@exa...| 17|2024-09-19|     Amy Chandler| 85082|
| 24|     Smithfurt|        Estonia|         IT|smitchell@example...| 19|2025-12-05|  Mr. James Brown|100217|
| 29|    Garciastad|North Macedonia|      Sales|bzimmerman@exampl...| 51|2023-04-24|Alexandria Fisher|122240|
| 39|    Wilsonland|           Iran| Operations|  ojones@example.net| 59|2016-07-04|     Vincent King|124012|
+---+--------------+---------------+-----------+--------

In [53]:
print("📌 Random sample: 20% (with replacement)")
df_sample_replace = df_large.sample(withReplacement=True, fraction=0.2, seed=42)
print(f"Sample size: {df_sample_replace.count()} rows")

📌 Random sample: 20% (with replacement)
Sample size: 204 rows


### 5.2: Limit

In [54]:
print("📌 Limit to first 5 rows")
df_large.limit(5).show()

📌 Limit to first 5 rows
+---+---------------+-----------+-----------+--------------------+---+----------+---------------+------+
|age|           city|    country| department|               email| id| join_date|           name|salary|
+---+---------------+-----------+-----------+--------------------+---+----------+---------------+------+
| 62|      East Jill|Philippines|Engineering|garzaanthony@exam...|  1|2018-02-27|   Allison Hill| 43278|
| 39|  Robinsonshire|       Fiji|      Sales|blakeerik@example...|  2|2019-05-17|Alyssa Gonzalez| 69256|
| 30|South Colinstad|Netherlands|         IT|lindsay78@example...|  3|2025-09-14|   Tyler Rogers| 53434|
| 65|    Barbaraland|      Aruba|         IT|kendragalloway@ex...|  4|2024-08-21|  Juan Calderon|111482|
| 27|   West Michael|   Paraguay|    Finance|jacqueline19@exam...|  5|2017-09-07|  Jesse Flowers| 95302|
+---+---------------+-----------+-----------+--------------------+---+----------+---------------+------+



### 5.3: Complex Queries (Chaining Operations)

In [55]:
print("📌 Query: High earners in Engineering")
result = df_large \
    .filter(col("department") == "Engineering") \
    .filter(col("salary") > 100000) \
    .select("name", "age", "salary", "city") \
    .orderBy(col("salary").desc())

print(f"Found {result.count()} high earners in Engineering")
result.show(10)

📌 Query: High earners in Engineering
Found 57 high earners in Engineering
+--------------------+---+------+----------------+
|                name|age|salary|            city|
+--------------------+---+------+----------------+
|Christopher Williams| 33|149602|    Lake William|
|        Nicole Lewis| 56|149392|       Sarahview|
|      Vanessa Foster| 34|148028|    Stewarthaven|
|        Krystal Diaz| 52|147756|      Susanburgh|
|      Sarah Phillips| 50|146295|       West John|
|       Wayne Russell| 50|146235|South Sherryberg|
|       Linda Wilkins| 64|145464|         Troyton|
|       Grace Swanson| 34|143692|    New Jennifer|
|       Wayne Perkins| 42|141259|      Barkerberg|
|         Sarah Jones| 59|140238|     West Jeremy|
+--------------------+---+------+----------------+
only showing top 10 rows



In [56]:
print("📌 Query: Average salary by department (top 5)")
df_large \
    .groupBy("department") \
    .agg(
        count("*").alias("employee_count"),
        avg("salary").alias("avg_salary")
    ) \
    .orderBy(col("avg_salary").desc()) \
    .limit(5) \
    .show()

📌 Query: Average salary by department (top 5)
+-----------+--------------+-----------------+
| department|employee_count|       avg_salary|
+-----------+--------------+-----------------+
| Operations|           137|99357.65693430656|
|  Marketing|           143|97774.46153846153|
|         IT|           151|96760.65562913907|
|         HR|           142|96117.23943661971|
|Engineering|           139|94520.53237410072|
+-----------+--------------+-----------------+



---

## ✅ SUMMARY

### 📚 What you learned:

**1️⃣ Spark Architecture:**
- Driver, Cluster Manager, Executors
- How jobs are distributed

**2️⃣ Creating DataFrames:**
- From list of tuples
- From list of dictionaries
- With explicit schema (recommended)

**3️⃣ Basic Operations:**
- Inspecting: show, printSchema, count, describe
- Selecting: select, col
- Filtering: filter, multiple conditions
- Adding columns: withColumn
- Modifying: withColumnRenamed
- Dropping: drop
- Sorting: orderBy, asc, desc
- Distinct: distinct, dropDuplicates

**4️⃣ Working with Data:**
- Generated 1000 rows with Faker
- Sampling
- Limiting
- Complex queries with chaining

### 🎯 Key Takeaways:
- ✅ DataFrame is immutable (operations return new DataFrame)
- ✅ Use col() for complex expressions
- ✅ Chain operations for readability
- ✅ Always check schema with printSchema()

### 📝 Next: Transformations vs Actions & Lazy Evaluation